In [54]:
#imports
import time
import qcportal as ptl
import numpy as np
import matplotlib.pyplot as plt
from fragmenter.utils import HARTREE_2_KJMOL
from simtk import unit
import os, sys
from fragmenter import chemi
import cmiles
import qcportal as ptl
from openforcefield.topology import Molecule, Topology
from openforcefield.typing.engines.smirnoff import ForceField


In [102]:
#functions 

def torsion_barrier_for_molecule(tdr_object, smiles, show_plots=False):
    """
    Takes in a single torsion drive record that has energies from multiple conformers (at different torsion angles), evaluates the torsion barrier 
    
    Parameters
    ----------
    tdr_object : object
        torsion drive record from QC archive for a molecule
    
    Returns
    -------
    tdr_object.id : int
        id of the TD record
    dihedral_indices: list
        list of atom indices for which torsion is driven in this record
    torsion_barrier: float
        torsion barrier energy in KJ/mol, maximum of all the barriers
    mol: oemol object
        oemol from the smiles in dataframe index
    """
    energies = list(tdr_object.get_final_energies().values())
    tmp = list(tdr_object.get_final_energies().keys())
    angles = [i[0]*np.pi/180 for i in tmp]
    angles, energies = zip(*sorted(zip(angles, energies)))
    angles = np.array(angles)
    energies = np.array(energies)
    angles = np.append(angles[-3:] - 2*np.pi, np.append(angles, angles[:3] + 2*np.pi))
    energies = np.append(energies[-3:], np.append(energies, energies[:3]))
    
    idx = []
    for i in range(len(angles)-2):
        m1 = (energies[i+1]-energies[i])/(angles[i+1]-angles[i])
        m2 = (energies[i+2]-energies[i+1])/(angles[i+2]-angles[i+1])
        if np.sign(m1) == np.sign(m2):
            continue
        else:
            idx.append(i+1)
        
    if (show_plots):
        min_ener = min(energies)
        energies_y = (energies-min_ener)*HARTREE_2_KJMOL
        fontsize = 14
        plt.figure()
        plt.plot(angles*180/np.pi, energies_y, 'b-X', angles[idx]*180/np.pi, energies_y[idx], 'ro')
        plt.legend(['QM data', 'Max, min'], bbox_to_anchor=(1, 1), fontsize=fontsize)
        plt.title('Torsion drive interpolation', fontsize=fontsize)
        plt.xlabel('Dihedral Angle [Degrees]', fontsize=fontsize)
        plt.ylabel('Relative energy [KJ / mol]', fontsize=fontsize)
        plt.xticks(fontsize=fontsize)
        plt.yticks(fontsize=fontsize)
        fig_name = 'plot_' + tdr_object.id + '.png'
        plt.savefig(fig_name)
        plt.show()
        
    torsion_barriers = []
    for i in range(int(len(idx)-1)):
        torsion_barriers.append(abs(HARTREE_2_KJMOL*abs(energies[idx[i]] - energies[idx[i+1]])))
    torsion_barriers = np.array(torsion_barriers)
    
    # get dihedral indices and pass on to get_wbo function
    dihedral_indices = list(tdr_object.dict()['keywords']['dihedrals'][0])
    mol = chemi.smiles_to_oemol(smiles)
    mol.SetData("TB", max(torsion_barriers))
    mol.SetData("TDindices", dihedral_indices)
    mol.SetData("TDid", tdr_object.id)
    mol.SetData("cmiles", smiles)
    return mol
    #return(tdr_object.id, dihedral_indices, max(torsion_barriers), mol)


def analyzeTorsion(tID, ff, k1, k2, qcaDS, fileName):
        """
        Description:
        This function takes in a torsion id, and performs analysis on it using a QCA torsion drive dataset
        This function will generate a plot with a .pdf extension

        Input:
        tID: String of a torsion ID from an .offxml format force field e.g. "t43"
        ff: file name of a .offxml style force field
        qcaDS: String of a name of a QCA torsion drive dataset
        fileName: String for the .pdf file name of the plot of WBO versus torsion drive barrier
        k1: Float of the force constant for wiberg bond order equal to 1 for the specified torsion
        k2: Float of the force constant for wiberg bond order equal to 2 for the specified torsion

        Return:
        resultDict: A dictionary of the results from the residual calculation. The key is the smiles and the
        items is a list of the [torsion barrier hieght, WBO, residual].
        """

        #WIP Load in the QCA dataset, convert the dataset into OEMols with data tags containing WBO and torsion barrier height
        molList = loadDataset(qcaDS)


        molsTorsion=[]
        for mol in molList:
            status = checkTorsion(tID, ff, mol)
            if status == True:
                molsTorsion.append(mol)

        resultDict={}

        for m in molsTorsion:
            resultDict[OEMolsToSmiles(m)]= [m.GetData("TorsionBarrier"), m.GetData("WBO"), getResidual(m, k1, k2)]

        plotResults(resultDict, fileName, k1, k2)


        return resultDict

def loadDataset(datasets):
    """
        To load torsion drive datasets and return a list of molecules with torsion barriers and associated wiberg bond order
        
        Parameters
        ----------
        datasets : list of tuples containing the dataset name and specification
        
        Returns
        -------
        molList : each row contains the tdr_object.id, oemol_object, dihedral_indices, torsion_barrier, wbo
    """
    molList = []
    for dataset_tuple in datasets:
        print("Reading dataset: ", dataset_tuple[0], ", and specification: ", dataset_tuple[1])
        molList.extend(loadDataset_low(dataset_tuple[0], dataset_tuple[1]))

    return molList
    
    

    

def loadDataset_low(datasetName, specification):
    """
    Low level call to load each torsion drive dataset and return a list of molecules
    
        Parameters
        ----------
        datasetName : str
            torsion drive dataset name.  
        specification : str
            specification in the dataset. Example: "B3LYP-D3", "default", "UFF" 
        
        Returns
        -------
        molList : list of objects
            each row contains the tdr_object.id, dihedral_indices, torsion_barrier, oemol_object
    """
    while True:    
        try:
            assert(datasetName)
            break
        except AssertionError:
            print ("datasetName is empty. Check input list of dataset tuples")
            raise
    while True:    
        try:
            assert(specification)
            break
        except AssertionError:
            print ("specification is empty. Check input list of dataset tuples")
            raise

    # initiate qc portal instance
    client = ptl.FractalClient()    
    # from the TorsionDriveDataset collection picking up given datasetName
    ds = client.get_collection("TorsionDriveDataset", datasetName)
    ds.status([specification], status="COMPLETE")
 
    # Serial implementation
    tb = []
    for i in range(ds.df.size):
        if (ds.df.iloc[i,0].status == 'COMPLETE'):
            smiles = ds.df.index[i]
            cmiles_ids = cmiles.get_molecule_ids(smiles, strict=False)
            mapped_smiles = cmiles_ids['canonical_isomeric_explicit_hydrogen_mapped_smiles']
            tb.append(torsion_barrier_for_molecule(ds.df.iloc[i, 0], mapped_smiles))
    print("Total records processed for this dataset:", len(tb), "out of ", len(ds.df))
    return(tb)





def getResidual(oemol, k1, k2):
    """
    Description:
    Finds the residual of an oemol by taking the input k1 and k2 values and finding the y=mx+b form and calculates
    the difference from the wbo, torsion barrier height point

    input:
    oemol: OEMol from oechem, contains datatags for WBO and torsion barrier height
    k1: Float of the force constant for wiberg bond order equal to 1 for the specified torsion
    k2: Float of the force constant for wiberg bond order equal to 2 for the specified torsion
    """
    #WIP

    return residual

def plotResult(resultDict, filename, k1, k2):
    """
    Description:
    Creates a scatter plot of WBO versus the torsion barrier height with the y=mx + b form of the torsion interpolation.
    Saves plot to .pdf

    Input:
    resultDict: Dictionary of the data for plotting. The keys are smiles, and the items are a list of WBO, torsion barrier height
    and residual
    fileName: Name of file to save .pdf form of the plot
    k1: Float of the force constant for wiberg bond order equal to 1 for the specified torsion
    k2: Float of the force constant for wiberg bond order equal to 2 for the specified torsion

    return:
    """
    #WIP
    return


def checkTorsion(molList, TID):
    matches=[]
    count=0
    #print(len(molList))
    for mol in molList: 
        #molecule = Molecule(mol, allow_undefined_stereo=True)
        molecule = Molecule.from_mapped_smiles(mol.GetData("cmiles"))
        #topology=Topology._from_openeye(mol)
        topology = Topology.from_molecules([molecule])
        # Let's label using the Parsley force field
        forcefield = ForceField('openff_unconstrained-1.2.1.offxml')
        # Run the molecule labeling
        molecule_force_list = forcefield.label_molecules(topology)
        params=[]
        # Print out a formatted description of the torsion parameters applied to this molecule
        for mol_idx, mol_forces in enumerate(molecule_force_list):
            #print(f'Forces for molecule {mol_idx}')
            for force_tag, force_dict in mol_forces.items():
                if force_tag == 'ProperTorsions':
                    #print(f"\n{force_tag}:")
                    
                    for (atom_indices, parameter) in force_dict.items():
                        #atomstr=''
                        atomList=[]
                        for idx in atom_indices:
                            atomList.append(idx)
                            #atomstr += '%3s' % idx
                        #print(atomList)
                        #print(mol.GetData("TDindices"))
                        params.append(parameter.id)
                        if atomList == mol.GetData("TDindices"):
                            count+=1
                            print(type(parameter.id))
                            print(parameter.id)
                            if parameter.id == TID:
                                print("Match!")
                                matches.append(mol)
                        #print("atoms: %s  parameter_id: %s  smirks %s" % (atomstr, parameter.id, parameter.smirks) )
    print(count)
    print(len(molList))
    return matches, params


In [103]:
#datasetName="OpenFF Gen 2 Torsion Set 5 Bayer"
datasetName="OpenFF Rowley Biaryl v1.0"
molList=loadDataset_low(datasetName, "default")
#print(molList)

#mol.SetData("TB", max(torsion_barriers))
#    mol.SetData("TDindices", dihedral_indices)
#    mol.SetData("TDid", tdr_object.id)
    
#for mol in molList:
    #print(mol.GetData("TDindices"))

mols=checkTorsion(molList, 't43')
print(mols)





RDKit ERROR: [10:26:47] Can't kekulize mol.  Unkekulized atoms: 6 9 10 11 12 13 15
RDKit ERROR: 
RDKit ERROR: [10:26:47] Can't kekulize mol.  Unkekulized atoms: 6 9 10 11 12 13 15
RDKit ERROR: 
RDKit ERROR: [10:26:47] Can't kekulize mol.  Unkekulized atoms: 6 9 10 11 12 13 15
RDKit ERROR: 
RDKit ERROR: [10:26:47] Can't kekulize mol.  Unkekulized atoms: 6 9 10 11 12 13 15
RDKit ERROR: 
RDKit ERROR: [10:26:47] Can't kekulize mol.  Unkekulized atoms: 6 9 10 11 12 13 15
RDKit ERROR: 


Total records processed for this dataset: 86 out of  87
<class 'str'>
t43
Match!
<class 'str'>
t43
Match!
<class 'str'>
t43
Match!
<class 'str'>
t43
Match!
<class 'str'>
t47
<class 'str'>
t47
<class 'str'>
t47
<class 'str'>
t43
Match!
<class 'str'>
t43
Match!
<class 'str'>
t43
Match!
<class 'str'>
t43
Match!
<class 'str'>
t43
Match!
<class 'str'>
t47
<class 'str'>
t47
<class 'str'>
t43
Match!
<class 'str'>
t43
Match!
<class 'str'>
t43
Match!
<class 'str'>
t43
Match!
<class 'str'>
t43
Match!
<class 'str'>
t43
Match!
<class 'str'>
t43
Match!
<class 'str'>
t43
Match!
<class 'str'>
t43
Match!
<class 'str'>
t43
Match!
<class 'str'>
t43
Match!
<class 'str'>
t43
Match!
26
86
([<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x13a1e41e0> >, <oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x13a1e4960> >, <oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x13a1e4060> >, <oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x13a1e49f0> 

In [ ]:
import qcportal as ptl
from openforcefield.topology import Molecule, Topology
from openforcefield.typing.engines.smirnoff import ForceField


client = ptl.FractalClient()
ds = client.get_collection("TorsionDriveDataset", "OpenFF Fragmenter Phenyl Benchmark")
ds.status("B3LYP-D3", status="COMPLETE")
# output = torsion_barrier_for_molecule(ds.df.iloc[1, 0], ds.df.index[1], True)
# print("Torsionderive record Id, Dihedral_indices, Torsion barrier, oemol")
# print(output)
dihedral_indices = list(ds.df.iloc[1, 0].dict()['keywords']['dihedrals'][0])
print("Dihedral indices for which torsion is driven: ", dihedral_indices)
molecule = Molecule.from_smiles(ds.df.index[1])
display(molecule.visualize)
topology = Topology.from_molecules([molecule])
# Let's label using the Parsley force field
forcefield = ForceField('openff-1.2.1.offxml')
# Run the molecule labeling
molecule_force_list = forcefield.label_molecules(topology)
# Print out a formatted description of the torsion parameters applied to this molecule
for mol_idx, mol_forces in enumerate(molecule_force_list):
    print(f'Forces for molecule {mol_idx}')
    for force_tag, force_dict in mol_forces.items():
        if force_tag == 'ProperTorsions':
            print(f"\n{force_tag}:")
            for (atom_indices, parameter) in force_dict.items():
                atomstr=''
                for idx in atom_indices:
                    atomstr += '%3s' % idx
                print("atoms: %s  parameter_id: %s  smirks %s" % (atomstr, parameter.id, parameter.smirks) )